In [500]:
import os
import os.path as osp
import pandas as pd
import numpy as np
from datetime import datetime
import ast
import re
from utils import helpers as hpr

### Load all changes

In [501]:
df = hpr.combine_openstack_data()
df = df[df['status']!='NEW']

Reading OpenStack changes...
OpenStack changes loaded successfully...


In [ ]:
df['messages'] = df.loc[df['messages'].notna(), 'messages'].map(ast.literal_eval)

In [4]:
df['Year'] = df['created'].map(lambda x: x.year)

### The median of changes per day

In [ ]:
# Convert 'creation_date' to string format 'year-month-day'
df['created_str'] = df['created'].dt.strftime('%Y-%m-%d')

# Group by 'creation_date' and count the number of rows per day
daily_counts = df.groupby(df['created'].dt.date).size()

# Display the result
daily_counts_df = daily_counts.reset_index(name='count')
daily_counts_df.columns = ['created_str', 'count']

daily_counts_df['count'].median()

In [410]:
df_build_fialure = pd.read_csv(osp.join(".", "Files", "Metrics", "num_build_failures.csv"))

In [ ]:
pd.read_csv("./Results/first_model_perf.csv")

### The growth of dependent changes

In [130]:
df_dep_ned_cha = pd.read_csv(osp.join('Files', 'source_target_evolution.csv'))
deps_need_changes = df_dep_ned_cha['Source'].tolist() + df_dep_ned_cha['Target'].tolist()
df['Count'] = 0
df_reduced = df.loc[df['number'].isin(deps_need_changes), ['status', 'created', 'Count']]

df_reduced['Year'] = df_reduced['created'].map(lambda x: x.year)

# df_reduced = df_reduced.groupby('Year').count().reset_index(level=0)

In [131]:
df_merged = df_reduced[df_reduced['status']=='MERGED']
df_merged = df_merged.groupby('Year').count().reset_index(level=0)
df_merged['Status'] = 'MERGED'

df_abandoned = df_reduced[df_reduced['status']=='ABANDONED']
df_abandoned = df_abandoned.groupby('Year').count().reset_index(level=0)
df_abandoned['Status'] = 'ABANDONED'

df_new = df_reduced[df_reduced['status']=='NEW']
df_new = df_new.groupby('Year').count().reset_index(level=0)
df_new['Status'] = 'NEW'

In [132]:
df_reduced = pd.concat((df_merged[['Year', 'Status', 'Count']], df_abandoned[['Year', 'Status', 'Count']], df_new[['Year', 'Status', 'Count']]))

In [133]:
def calc_perc_dependencies(row):
    all_changes = df.loc[(df['status']==row['Status'])&(df['Year'] == row['Year']), 'number'].nunique()
    return round(100 * (row['Count'] / all_changes), 2)

In [17]:
df_reduced['percentage'] = df_reduced.apply(calc_perc_dependencies, axis=1)

In [74]:
df_reduced.to_csv("./Files/Preliminary/deps_evolution.csv", index=None)

### When dependent changes are identified

In [503]:
def time_diff(start, end):
    if start > end:
        start, end = end, start
    current_date =  datetime.strptime(end, "%Y-%m-%d %H:%M:%S") 
    previous_date = datetime.strptime(start, "%Y-%m-%d %H:%M:%S") 
    diff = current_date - previous_date
    diff = float("{:.2f}".format(diff.total_seconds() / 3600))
    return diff


def extract_attr(x, attr):
    '''Extracts the passed-on parameter values out of the commit message 
    '''
    rs = re.findall("%s:\s[a-zA-Z0-9/\.\:\+\-\#]{6,}" % (attr), x)
    result = []
    for row in rs:
        row = row[len(attr) + 2:]
        change_id_pattern = re.search(r"[a-zA-Z0-9]{41}", row)
        if change_id_pattern:
            result.append(change_id_pattern[0])
            continue
        number_pattern = re.search("#?https?[\:][/]{2}review[\.](opendev|openstack)[\.]org([a-z0-9A-Z\-\+/\.#]*)\d+", row)
        if number_pattern:
            result.append(int(re.search("\d+$", number_pattern[0][0:])[0]))
    return result if len(result) != 0 else None


def retrieve_revision_date(row, attr, return_revision_date=True):
    number = None
    second_number = None

    if attr == "Depends-On":
        number = row["Target"]
        second_number = row["Source"]
        change_id = row["Source_change_id"]
    else:
        number = row["Source"]
        second_number = row["Target"]
        change_id = row["Target_change_id"]

    df_row = df.loc[df["number"] == number]
    revisions = ast.literal_eval(df_row["revisions"].values[0])
    revisions = sorted(revisions, key=lambda x: x["created"])
    if  len(revisions) == 1:
        if return_revision_date:
            return revisions[0]["created"][:-11]
        else:
            return 1

    first_revision = revisions[0]
    first_message = first_revision["message"]

    results = extract_attr(first_message, attr)

    if results and ((change_id in results) or (second_number in results)):
        if return_revision_date:
            return first_revision["created"][:-11]
        else:
            return 1

    for i in range(1,len(revisions)):
        current_message = revisions[i]["message"]
        created = revisions[i]["created"]
        results = extract_attr(current_message, attr)
        
        if results and ((change_id in results) or (second_number in results)):

            if return_revision_date:
                return created[:-11]
            else:
                return i + 1

def is_same_developer(row):
    return "Same" if row["Source_dev"] == row["Target_dev"] else "Different"

def identify_dependency(row):
    source_date = row["Source_date"] 
    target_date = row["Target_date"]
    link_date = datetime.strptime(row["link_date"], "%Y-%m-%d %H:%M:%S")
    delta1 = (target_date - link_date).total_seconds() / (60 * 60)
    delta2 = (source_date - link_date).total_seconds() / (60 * 60)

    return abs(delta2)
    # return min(abs(delta1), abs(delta2))

#### Depends-On

In [502]:
all_changes = df['number'].unique()

In [504]:
df_depends_on = pd.read_csv("./Files/source_target_depends.csv")
df_depends_on = df_depends_on[df_depends_on['Source'].isin(all_changes)&df_depends_on['Target'].isin(all_changes)]
df_depends_on["Source_status"] = df_depends_on["Source"].map(lambda x: df.loc[df["number"]==x, "status"].values[0])
df_depends_on["Target_status"] = df_depends_on["Target"].map(lambda x: df.loc[df["number"]==x, "status"].values[0])

df_depends_on["revisions"] = df_depends_on["Target"].map(lambda x: df.loc[df["number"]==x, "revisions"].values[0])
df_depends_on["Source_change_id"] = df_depends_on["Source"].map(lambda x: df.loc[df["number"]==x, "change_id"].values[0])
df_depends_on["Target_change_id"] = df_depends_on["Target"].map(lambda x: df.loc[df["number"]==x, "change_id"].values[0])

df_depends_on["link_date"] = df_depends_on.apply(retrieve_revision_date, args=("Depends-On",), axis=1)

df_depends_on["worked_revisions"] = df_depends_on.apply(retrieve_revision_date, args=("Depends-On",False,), axis=1)

df_depends_on["is_cross"] = df_depends_on.apply(lambda row: "Cross" if row["Source_repo"]!=row["Target_repo"] else "Same", axis=1)

df_depends_on["is_source_bot"] = df_depends_on["Source"].map(lambda x: df.loc[df["number"]==x,"is_owner_bot"].values[0])
df_depends_on["is_target_bot"] = df_depends_on["Target"].map(lambda x: df.loc[df["number"]==x,"is_owner_bot"].values[0])

df_depends_on["Source_dev"] = df_depends_on["Source"].map(lambda x: df.loc[df["number"]==x,"owner_account_id"].values[0])
df_depends_on["Target_dev"] = df_depends_on["Target"].map(lambda x: df.loc[df["number"]==x,"owner_account_id"].values[0])

df_depends_on["same_dev"] = df_depends_on.apply(is_same_developer, axis=1)

df_depends_on["Source_date"] = df_depends_on["Source"].map(lambda x: df.loc[df["number"]==x, "created"].values[0])
df_depends_on["Target_date"] = df_depends_on["Target"].map(lambda x: df.loc[df["number"]==x, "created"].values[0])

df_depends_on["when_identified"] = df_depends_on[["Source_date", "Target_date", "link_date"]].apply(identify_dependency, axis=1)

df_depends_on = df_depends_on[(df_depends_on['Source_status']=='MERGED')|(df_depends_on['Target_status']=='MERGED')]


#### Needed-By

In [505]:
df_needed_by = pd.read_csv("./Files/source_target_needed.csv")
df_needed_by = df_needed_by[df_needed_by['Source'].isin(all_changes)&df_needed_by['Target'].isin(all_changes)]
df_needed_by["Source_status"] = df_needed_by["Source"].map(lambda x: df.loc[df["number"]==x, "status"].values[0])
df_needed_by["Target_status"] = df_needed_by["Target"].map(lambda x: df.loc[df["number"]==x, "status"].values[0])

df_needed_by["revisions"] = df_needed_by["Source"].map(lambda x: df.loc[df["number"]==x, "revisions"].values[0])
df_needed_by["Source_change_id"] = df_needed_by["Source"].map(lambda x: df.loc[df["number"]==x, "change_id"].values[0])
df_needed_by["Target_change_id"] = df_needed_by["Target"].map(lambda x: df.loc[df["number"]==x, "change_id"].values[0])

df_needed_by["link_date"] = df_needed_by.apply(retrieve_revision_date, args=("Needed-By",), axis=1)
df_needed_by["worked_revisions"] = df_needed_by.apply(retrieve_revision_date, args=("Needed-By",False,), axis=1)

df_needed_by["is_cross"] = df_needed_by.apply(lambda row: "Cross" if row["Source_repo"]!=row["Target_repo"] else "Same", axis=1)

df_needed_by["is_source_bot"] = df_needed_by["Source"].map(lambda x: df.loc[df["number"]==x,"is_owner_bot"].values[0])
df_needed_by["is_target_bot"] = df_needed_by["Target"].map(lambda x: df.loc[df["number"]==x,"is_owner_bot"].values[0])

df_needed_by["Source_dev"] = df_needed_by["Source"].map(lambda x: df.loc[df["number"]==x,"owner_account_id"].values[0])
df_needed_by["Target_dev"] = df_needed_by["Target"].map(lambda x: df.loc[df["number"]==x,"owner_account_id"].values[0])

df_needed_by["same_dev"] = df_needed_by.apply(is_same_developer, axis=1)

df_needed_by["Source_date"] = df_needed_by["Source"].map(lambda x: df.loc[df["number"]==x, "created"].values[0])
df_needed_by["Target_date"] = df_needed_by["Target"].map(lambda x: df.loc[df["number"]==x, "created"].values[0])

df_needed_by["when_identified"] = df_needed_by[["Source_date", "Target_date", "link_date"]].apply(identify_dependency, axis=1)
df_needed_by = df_needed_by[(df_needed_by['Source_status']=='MERGED')|(df_needed_by['Target_status']=='MERGED')]

In [557]:
dependency_identification = pd.concat((df_depends_on, df_needed_by)).sort_values("when_identified")
dependency_identification = dependency_identification.drop_duplicates(subset=["Source", "Target"], keep="first")

### Calculating the lag

In [551]:
def time_diff(row):
    start, end = row['Source_date'], row['Target_date']
    if start > end:
        start, end = end, start
    # current_date =  datetime.strptime(end, "%Y-%m-%d %H:%M:%S") 
    # previous_date = datetime.strptime(start, "%Y-%m-%d %H:%M:%S") 
    diff = end - start
    diff = float("{:.2f}".format(diff.total_seconds() / 3600))
    return diff

def lag_nbr_changes(row):
    start, end = row['Source_date'], row['Target_date']
    if start > end:
        start, end = end, start
    end =  datetime.strptime(end, "%Y-%m-%d %H:%M:%S") 
    start = datetime.strptime(start, "%Y-%m-%d %H:%M:%S") 
    res = df.loc[(df['created']>=start)&(df['created']<=end), "number"].nunique()
    return res

In [558]:
dependency_identification['lag'] = dependency_identification.apply(time_diff, axis=1)

In [554]:
dependency_identification = dependency_identification[['lag']]

# Calculate Z-scores
z_scores = np.abs((dependency_identification - dependency_identification.mean()) / dependency_identification.std())

# Set a threshold for identifying outliers
threshold = 3

# Filter out the outliers
df_clean = dependency_identification[(z_scores < threshold).all(axis=1)]

In [561]:
len(df_clean.loc[df_clean['lag']<=38*24, "lag"])/len(df_clean)

0.8572188404940606

In [564]:
df_clean['lag'].max()

6315.89

In [562]:
# len(dependency_identification.loc[dependency_identification['lag']<=38*24, "lag"])/len(dependency_identification)
dependency_identification['lag'].max()

61999.54

In [547]:
# dependency_identification['lag_changes'] = dependency_identification.apply(lag_nbr_changes, axis=1)

In [ ]:
dependency_identification.loc[dependency_identification['lag']==57, 'lag_changes']

### When miss a dependency

In [492]:
after_revi_deps = df_depends_on.loc[df_depends_on['when_identified']>1, 'Source'].unique()
after_revi_need = df_needed_by.loc[(~df_needed_by['Target'].isin(after_revi_deps))&(df_needed_by['when_identified']>1), 'Target'].unique()
after_review = set(after_revi_deps.tolist()+after_revi_deps.tolist())

all_deps_deps_on = df_depends_on['Source'].unique()
all_deps_need = df_needed_by.loc[(~df_needed_by['Target'].isin(all_deps_deps_on)), 'Target'].unique()
all_dependent_changes = set(all_deps_deps_on.tolist()+all_deps_need.tolist())

In [493]:
df_depends_ident = df_depends_on.loc[(df_depends_on['Source'].isin(after_revi_deps)), "when_identified"].tolist()
df_needed_ident = df_needed_by.loc[(df_needed_by['Target'].isin(after_revi_need)), "when_identified"].tolist()
ident_time = df_depends_ident + df_needed_ident

In [ ]:
np.max(ident_time)

In [392]:
df_depends_ident = df_depends_on.loc[(df_depends_on['worked_revisions']>1)&(df_depends_on['when_identified']>0)&(df_depends_on['Source'].isin(after_revi_deps)), "when_identified"].tolist()
df_needed_ident = df_needed_by.loc[(df_needed_by['worked_revisions']>1)&(df_needed_by['when_identified']>0)&(df_needed_by['Target'].isin(after_revi_need)), "when_identified"].tolist()
after_rev_ident_time = df_depends_ident + df_needed_ident

In [398]:
pd.DataFrame({"time": after_rev_ident_time}).to_csv(osp.join(".", "Files","Preliminary", "deps_ident.csv"), index=None)

In [131]:
all_dependent_changes = pd.concat((df_depends_on, df_needed_by))
all_dependent_changes.drop_duplicates(subset=['Source', 'Target'], inplace=True)
all_dependent_changes = set(hpr.flatten_list(all_dependent_changes[['Source', 'Target']].values))

In [360]:
len(df[(df['status']=='MERGED')&df['number'].isin(after_review)])/len(df[df['number'].isin(all_dependent_changes)&(df['status']=='MERGED')])

0.2732972874733064

In [459]:
dependency_identification.loc[(dependency_identification['worked_revisions']>1)&(dependency_identification['worked_revisions']>1), "when_identified"].max()

17336.070277777777

In [95]:
test = pd.concat((df_depends_on, df_needed_by))
test.drop_duplicates(subset=['Source', 'Target'], inplace=True)

In [103]:
len(set(hpr.flatten_list(test[['Source', 'Target']].values)))

49000

In [137]:
dependency_identification.to_csv(osp.join(".", "Files", "Preliminary", "deps_ident.csv"), index=False)

In [109]:
dependency_identification = pd.read_csv(osp.join(".", "Files", "Preliminary", "deps_ident.csv"))

In [110]:
dependency_identification.loc[dependency_identification['iden_type']=='slow', 'when_identified'].median()/24

91.7265162037037

In [32]:
# dependency_identification['when_identified'].median()
dependency_identification.loc[(dependency_identification['Source_status']=='MERGED')&(dependency_identification['Target_status']=='MERGED')].to_csv(osp.join(".", "Files", "Preliminary", "deps_ident.csv"), index=False)

### How many dependent changes contain a build failure

In [411]:
df_dependent_cha = pd.DataFrame({'number': list(all_dependent_changes)})
df_dependent_cha = pd.merge(
    df_dependent_cha,
    right=df_build_fialure,
    on='number'
)

In [413]:
len(df_dependent_cha[(df_dependent_cha['num_build_failures']>1)&df_dependent_cha['number'].isin(after_review)])/len(df_dependent_cha[(df_dependent_cha['num_build_failures']>1)])

0.5158020274299344

In [404]:
def has_build_failure(nbr):
    res = df.loc[df['number']==nbr, 'messages'].tolist()[0]
    # print(res)
    return type(res) == list

def when_deps_linked(row):
    link_date_deps = df_depends_on.loc[df_depends_on['Target']==row['number'], 'link_date'].tolist()
    link_date_need = df_needed_by.loc[df_needed_by['Source']==row['number'], 'link_date'].tolist()
    res = link_date_deps + link_date_need
    res.sort()
        
    return res if len(res) > 0 else None

def when_build_failure_occur(row):
    linked_date = row['when_linked'][0]
    messages = row['messages']
    messages = sorted(messages, key=lambda d: d['date'])
    build_fail_date = messages[0]['date'][:18]
    print(linked_date, build_fail_date)
    if linked_date == build_fail_date:
        return 'Instant'
    elif linked_date > build_fail_date:
        return 'Before'
    else:
        return 'After'

In [ ]:
df_dependent_cha['has_build_failure'] = df_dependent_cha['number'].map(has_build_failure)

In [ ]:
df_dependent_cha = pd.merge(
    left=df_dependent_cha, 
    right=df[['number', 'revisions', 'messages']], 
    left_on='number', 
    right_on='number', 
    how='inner',
    suffixes=('_target', '_source')
)

In [192]:
df_dependent_cha["revisions"] = df_dependent_cha.loc[df_dependent_cha['when_linked'].notna(), "revisions"].map(ast.literal_eval)

In [162]:
len(df_dependent_cha[(df_dependent_cha['has_build_failure']==True)])/len(df_dependent_cha)

0.6457274105788957

In [193]:
df_dependent_cha['when_linked'] = df_dependent_cha[df_dependent_cha['has_build_failure']==True].apply(when_deps_linked, axis=1)

In [ ]:
df_dependent_cha['when_failure_occur'] = df_dependent_cha[df_dependent_cha['when_linked'].notna()].apply(when_build_failure_occur,axis=1)

In [205]:
7179/len(df_dependent_cha.loc[df_dependent_cha['when_linked'].notna(), "when_failure_occur"])#.value_counts())

0.4076660988074957

### What the size of our testing set

In [566]:
result = []
for i in range(10):
    test = pd.read_csv(f"./Files/Data/Test/{i}.csv")
    result.append(len(test))

### Manual analysis

In [ ]:
df_manual_analysis = pd.DataFrame({'number': df_depends_on['Target'].unique().tolist() + df_needed_by['Source'].unique().tolist()})
df_manual_analysis.drop_duplicates(subset='number', inplace=True)
df_manual_analysis = df_manual_analysis.sample(n=379)
df_manual_analysis.sort_values('number', inplace=True)
df_manual_analysis.reset_index(drop=True, inplace=True)

df_manual_analysis = pd.merge(
    left=df_manual_analysis, 
    right=df_depends_on.drop_duplicates(subset='Target')[['Target', 'worked_revisions']], 
    left_on='number',
    right_on='Target',
    how='left',
    suffixes=('_target', '_source')
)
df_manual_analysis.loc[df_manual_analysis['worked_revisions'].isnull(), 'worked_revisions'] = df_manual_analysis.loc[df_manual_analysis['worked_revisions'].isnull(), 'number'].map(lambda x: df_needed_by.loc[df_needed_by['Source']==x, 'worked_revisions'].values[0])
labels = {
    1: "Configuration",
    2: "Dependency",
    3: "Code enhancement",
    4: "New features",
    5: "Docs",
    6: "Renaming",
    7: "Tests",
    8: "Refactoring",
    9: "Moving resources",
    10: "Others",
}
nbr = 920233
df_manual_analysis.loc[
    df_manual_analysis['number']==nbr,
    ["reviewer_suggest", "reason", 'comment']
] = [0, labels[2], ""]
df_manual_analysis.to_csv(osp.join(".", "Files", "manual_analysis.csv"), index=None)

In [55]:
df_manual_analysis = pd.read_csv(osp.join(".", "Files", "manual_analysis.csv"))

In [428]:
df_manual_analysis

,number,worked_revisions,reason,comment,reviewer_suggest,tag
0,156691,1.0,Refactoring,NaN,0.0,NaN
1,161355,1.0,Code enhancement,NaN,0.0,NaN
2,175394,1.0,Code enhancement,NaN,0.0,NaN
3,181260,1.0,Tests,NaN,0.0,NaN
4,182072,1.0,Configuration,NaN,0.0,NaN
...,...,...,...,...,...,...
374,918920,1.0,Tests,NaN,0.0,NaN
375,919265,1.0,Tests,NaN,0.0,NaN
376,919339,1.0,Tests,NaN,0.0,NaN
377,919494,1.0,Tests,NaN,0.0,NaN


In [188]:
df_manual_analysis = pd.read_csv(osp.join(".", "Files", "manual_analysis.csv"))
df_manual_analysis.loc[df_manual_analysis['reason']=='Test', 'reason'] = 'Tests'
df_manual_analysis['reason'] = df_manual_analysis['reason'].map(lambda x: x.split(','))
df_manual_analysis = df_manual_analysis.explode(column='reason')
# Calculate the percentage of missing dependencies
# df_manual_analysis = df_manual_analysis.groupby('reason').apply(lambda x: pd.Series({
#     '\# missing deps': (x['worked_revisions'] > 1).sum(),
#     'Total': x['worked_revisions'].count(),
#     '\\% of missing': f"{round((x['worked_revisions'] > 1).sum() / len(x) * 100, 2)}\%"
#     # 'Description': 'No description yet'
# })).reset_index()

# # # Rename columns to match the desired output
# df_manual_analysis.rename(columns={'reason': 'Category'}, inplace=True)
# df_manual_analysis.sort_values(by='\# missing deps', ascending=0, inplace=True)

In [310]:
tags = {
    1: 'Complex change',
    2: 'Project unfamiliarity',
    3: 'Inconsistent deps',
    4: 'Build failure'
}

In [186]:
print(df_manual_analysis.to_latex(index=False, caption='A detailed overview of various reasons for which developers miss adding a dependency.', label='tab:manual-analysis', float_format="%.2f", ))

\begin{table}
\caption{A detailed overview of various reasons for which developers miss adding a dependency.}
\label{tab:manual-analysis}
\begin{tabular}{lrrl}
\toprule
Category & \# missing deps & Total & \% of missing \\
\midrule
Configuration & 43 & 110 & 39.09\% \\
Dependency & 28 & 55 & 50.91\% \\
Refactoring & 26 & 60 & 43.33\% \\
New features & 25 & 48 & 52.08\% \\
Tests & 23 & 52 & 44.23\% \\
Code enhancement & 14 & 26 & 53.85\% \\
Docs & 8 & 13 & 61.54\% \\
Moving resources & 3 & 11 & 27.27\% \\
Renaming & 2 & 11 & 18.18\% \\
Duplicate & 0 & 1 & 0.0\% \\
\bottomrule
\end{tabular}
\end{table}

